In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Loading the dataset
df=pd.read_csv("/kaggle/input/heart-disease-uci/heart.csv")
df.head()

In [ ]:
## Loading Libaries
import pandas as pd
import numpy as np 

In [ ]:
## Ignoring the Warnings
import warnings
warnings.filterwarnings("ignore")

### As I am focusing on the hyperparameter tuning technique so I am skipping EDA and Feature Selection.

# Feature Extracting 

In [ ]:
# For independent features 
X=df.drop("target",axis=1)
X.head()

In [ ]:
# For Dependent features
y=df.loc[:,["target"]]
y.head()

# Train Test Split

In [ ]:
# Splitting the dataset and taking test size as 35%
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.35,random_state=50)
print(X_train.shape)
print(X_test.shape)

# Model building by traditional normal Random Forest Classifier

In [ ]:
# Normal RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
model_nor=RandomForestClassifier(n_estimators=5)
model_nor.fit(X_train,y_train)
y_prednor=model_nor.predict(X_test)

In [ ]:
# Checking the Confusion Matrix and Classification Report
from sklearn import metrics 
print(metrics.confusion_matrix(y_test,y_prednor))
print(metrics.classification_report(y_test,y_prednor))

#### As we have seen that the accuracy of the model is 76%. Let see by applying hyperparameter tuning how we will improve the accuracy of the model

# Hyperparameter Technique

## RandomSearchCV
### Random search is a technique where random combinations of the hyperparameters are used to find the best solution for the built model. It is similar to grid search, and yet it has proven to yield better results comparatively

In [ ]:
## Loading RandomSearchCV from sklearn library
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
## Creating Space 
n_estimators=[int(x) for x in np.linspace(100,1200,10)]
criterion=["gini","entropy"]
max_depth=[50,100,150,200,300,500]
min_samples_split=[int(x) for x in np.linspace(5,50,10)]
min_samples_leaf=[int(x) for x in np.linspace(4,60,10)]
max_features=["auto", "sqrt", "log2"]
space_random={"n_estimators":n_estimators,
             "criterion":criterion,
             "max_depth":max_depth,
             "min_samples_split":min_samples_split,
             "min_samples_leaf":min_samples_leaf,
             "max_features":max_features}
space_random

In [ ]:
# Fitting RandomSearchCV
model2=RandomForestClassifier()
random=RandomizedSearchCV(estimator=model2,param_distributions=space_random,n_iter=100,cv=5,verbose=2,n_jobs=-1,
                         random_state=100)
random.fit(X_train,y_train)

In [ ]:
# Find the best parameter with the help of RandomSearchcv
random.best_params_

In [ ]:
## Fitting the model (Random Forest Classifer) by taking the parameter of RandomSearchCv
model_random=RandomForestClassifier(n_estimators=833,max_depth=150,max_features="auto",
                                    criterion="entropy",min_samples_leaf=35,min_samples_split=30)
model_random.fit(X_train,y_train)
y_predrand=model_random.predict(X_test)

In [ ]:
# Finding accuracy and classification report 
print(metrics.confusion_matrix(y_test,y_predrand))
print(metrics.classification_report(y_test,y_predrand))

## As we have seen that the model accuracy has incresed by 7% that is 83%. Now we will try GridSearchCV to improve the model

# GridSearchCV

## Grid search is the process of performing hyper parameter tuning in order to determine the optimal values for a given model. This is significant as the performance of the entire model is based on the hyper parameter values specified.

In [ ]:
random.best_params_

In [ ]:
from sklearn.model_selection import GridSearchCV
# Creating space
n_estimators=[830,833,840]
min_samples_split=[30,35]
min_samples_leaf=[30,35,40]
max_features=["auto"]
max_depth=[150,160,170]
criterion=["entropy"]
space_grid={"n_estimators":n_estimators,
           "min_samples_split":min_samples_split,
           "min_samples_leaf":min_samples_leaf,
           "max_features":max_features,
           "max_depth":max_depth,
           "criterion":criterion}
space_grid

In [ ]:
# Fitting in GridSearchCv 
model3=RandomForestClassifier()
grid=GridSearchCV(estimator=model3,param_grid=space_grid,cv=5,verbose=2,n_jobs=-1)
grid.fit(X_train,y_train)

In [ ]:
# Best parameters
grid.best_params_

In [ ]:
# ## Fitting the model (Random Forest Classifer) by taking the parameter of GridSearchCV
model_grid=RandomForestClassifier(n_estimators=833,criterion="entropy",max_depth=160,max_features="auto",
                                 min_samples_leaf=35,min_samples_split=35)
model_grid.fit(X_train,y_train)
y_predgrid=model_grid.predict(X_test)

In [ ]:
# Checking the accuracy and Classification Report
print(metrics.confusion_matrix(y_test,y_predrand))
print(metrics.classification_report(y_test,y_predrand))

## As we see the model accuracy is same as 83%. so we can conclude that with the help of tunning the parameters we can easily increase the accuracy of the model 